<a href="https://colab.research.google.com/github/luguzman/Masterclass-en-Inteligencia-Artificial/blob/main/Clientes_perdidos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
import cv2
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import plotly.express as px
import plotly.graph_objects as go

from google.colab import files #Librería para cargar ficheros directamente en Colab
%matplotlib inline

In [2]:
# Necesitaremos montar su disco usando los siguientes comandos:
# Para obtener más información sobre el montaje, puedes consultar: https://stackoverflow.com/questions/46986398/import-data-into-google-colaboratory

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df_scaled = pd.read_excel('/content/drive/MyDrive/Proyectos/BBVA/Clientes perdidos/resultados3.xlsx')
df_scaled

,Unnamed: 0,BANCA EXPRESS,BANCA PATRIMONIAL,BANCA PERSONAL,BANCA PERSONAL PLUS,BANCA PRIVADA,BANCARIO ALTO,BANCARIO BAJO,BANCARIO ESTANDAR,MICRONEGOCIO,MICRONEGOCIO BAJO,MONO FINANZIA,NEGOCIO,PYME,PYME PLUS,PYME PLUS PLUS,NO TARGET,TARGET,CON NOMINA NO TARGET,CON NOMINA TARGET,SIN NOMINA NO TARGET,SIN NOMINA TARGET,FH_ULT_MES_ACTIVO,FH_MIN_NOM_DISP,FH_MAX_NOM_DISP,IX_ACT_PPIO_FINAL,IX_NOMINA,IX_PERDIDO
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,16.0,NaN,NaN,NaN,0,1
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,11.0,46.0,16.0,NaN,1,1
2,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0.0,NaN,NaN,NaN,0,1
3,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0.0,NaN,NaN,NaN,0,1
4,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,14.0,NaN,NaN,NaN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,399995,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0.0,0.0,0.0,0.0,1,0
399996,399996,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0.0,NaN,NaN,0.0,0,0
399997,399997,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0.0,0.0,0.0,0.0,1,0
399998,399998,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0.0,NaN,NaN,0.0,0,0


In [5]:
df_scaled.isna().sum()

Unnamed: 0                   0
BANCA EXPRESS                0
BANCA PATRIMONIAL            0
BANCA PERSONAL               0
BANCA PERSONAL PLUS          0
BANCA PRIVADA                0
BANCARIO ALTO                0
BANCARIO BAJO                0
BANCARIO ESTANDAR            0
MICRONEGOCIO                 0
MICRONEGOCIO BAJO            0
MONO FINANZIA                0
NEGOCIO                      0
PYME                         0
PYME PLUS                    0
PYME PLUS PLUS               0
NO TARGET                    0
TARGET                       0
CON NOMINA NO TARGET         0
CON NOMINA TARGET            0
SIN NOMINA NO TARGET         0
SIN NOMINA TARGET            0
FH_ULT_MES_ACTIVO            0
FH_MIN_NOM_DISP         166667
FH_MAX_NOM_DISP         166667
IX_ACT_PPIO_FINAL       200000
IX_NOMINA                    0
IX_PERDIDO                   0
dtype: int64

In [6]:
df_scaled.drop(['Unnamed: 0','FH_MIN_NOM_DISP','IX_ACT_PPIO_FINAL'], axis=1, inplace = True)

In [7]:
df_scaled.shape

(400000, 25)

In [8]:
# from keras.optimizers import SGD

# Glorot Uniform initializer: https://keras.rstudio.com/reference/initializer_glorot_uniform.html

input_df = Input(shape = (25,)) 
x = Dense(50, activation = 'relu')(input_df)
x = Dense(500, activation = 'relu', kernel_initializer = 'glorot_uniform')(x)
x = Dense(500, activation = 'relu', kernel_initializer = 'glorot_uniform')(x)
x = Dense(2000, activation = 'relu', kernel_initializer = 'glorot_uniform')(x)
# Hasta este punto habré complicado habré mezclado, repartido las vasriables en tanta densidad durante la fase de codificación para al final 
# quedarme con un resumen de solamente 8 variables que serán muchisimas menos que las 37 de partida que teníamos. Estas 8 variables
# van a tener que resumir la misma información que tenían las de partida.
encoded = Dense(8, activation = 'relu', kernel_initializer = 'glorot_uniform')(x)
# Tenemos que re-crear, reconstruir la info. Observemos que no es simétrica, que le he quitado una capa de 500, esto para evitar debido a la 
# simetría, que podría copiar el autoencoder los pesos de entrada a los de salida "podría hacer trampa" y en base a pasar toda esa info que
# no convergiera del mejor modo.
x = Dense(2000, activation = 'relu', kernel_initializer = 'glorot_uniform')(encoded)
x = Dense(500, activation = 'relu', kernel_initializer = 'glorot_uniform')(x)
decoded = Dense(25, kernel_initializer = 'glorot_uniform')(x)

# autoencoder: Este solo lo vamos a utilizar para el entrenamiento de tanto los pesos del codificador como el descodificador,--> 
autoencoder = Model(input_df, decoded)

# -->Sin embargo, recordemos que aquí mi problema es reducir la dimensión, quedarme con la versión codificada e intermedia para tener menos datos 
# y poderle aplicar un k-means posteriormente. Es por eso que además del propio autoencoder me quedaré con una referencia solo al encoder que
# vuelve a ser un modelo, pero que desde la capa de entrada se queda sobre la codificada
# encoder - utilizado para reducir la dimensión
encoder = Model(input_df, encoded)

autoencoder.compile(optimizer = 'adam', loss='mean_squared_error') # Utilizamos mean_squared_error ya que ahora estamos en valores continuos

In [ ]:
# Entonces realizamos el entrenamiento de la red dodne la entrada y la salida son la misma (sales_df_scaled)
# Nota: Podríamos usar el scaled o el normal no hay diferencia en el autoencoder 
autoencoder.fit(df_scaled, df_scaled, batch_size = 128, epochs = 500, verbose = 3)

Epoch 1/500
Epoch 2/500
Epoch 3/500
Epoch 4/500
Epoch 5/500
Epoch 6/500
Epoch 7/500
Epoch 8/500
Epoch 9/500
Epoch 10/500
Epoch 11/500
Epoch 12/500
Epoch 13/500
Epoch 14/500
Epoch 15/500
Epoch 16/500
Epoch 17/500
Epoch 18/500
Epoch 19/500
Epoch 20/500
Epoch 21/500
Epoch 22/500
Epoch 23/500
Epoch 24/500
Epoch 25/500
Epoch 26/500
Epoch 27/500
Epoch 28/500
Epoch 29/500
Epoch 30/500
Epoch 31/500
Epoch 32/500
Epoch 33/500
Epoch 34/500
Epoch 35/500
Epoch 36/500
Epoch 37/500
Epoch 38/500
Epoch 39/500
Epoch 40/500
Epoch 41/500
Epoch 42/500
Epoch 43/500
Epoch 44/500
Epoch 45/500
Epoch 46/500
Epoch 47/500
Epoch 48/500
Epoch 49/500
Epoch 50/500
Epoch 51/500
Epoch 52/500
Epoch 53/500
Epoch 54/500
Epoch 55/500
Epoch 56/500
Epoch 57/500
Epoch 58/500
Epoch 59/500
Epoch 60/500
Epoch 61/500
Epoch 62/500
Epoch 63/500
Epoch 64/500
Epoch 65/500
Epoch 66/500
Epoch 67/500
Epoch 68/500
Epoch 69/500
Epoch 70/500
Epoch 71/500
Epoch 72/500
Epoch 73/500
Epoch 74/500
Epoch 75/500
Epoch 76/500
Epoch 77/500
Epoch 78

In [ ]:
pred = encoder.predict(df_scaled)